**LIGNÉES CELLULAIRES COMMUNES DANS LES DIFFÉRENTS FICHIERS DU DATASET CCLE**

In [1]:
import pandas as pd
from functools import reduce

# Liste des fichiers et des noms de colonnes à extraire
files = [
    "cell_CN_raw.tsv",
    "cell_exp_raw.tsv",
    "cell_mut_raw.tsv",
    "drug_screening_matrix_ccle.tsv",
    "drug_screening.tsv"
]

# Dictionnaire pour stocker les ensembles de noms de lignées cellulaires
cell_lines_sets = {}

# Lecture des fichiers et extraction des noms de lignées cellulaires
for file in files:
    try:
        df = pd.read_csv(file, sep="\t", dtype=str, low_memory=False)  # Correction ici
        column_name = "CELL_LINE_NAME" if "CELL_LINE_NAME" in df.columns else "cell_line_name"
        if column_name in df.columns:
            cell_lines_sets[file] = set(df[column_name].dropna().unique())
        else:
            print(f"Warning: Column 'cell_line_name' or 'CELL_LINE_NAME' not found in {file}")
    except Exception as e:
        print(f"Error reading {file}: {e}")

# Trouver toutes les lignées cellulaires uniques
all_unique_cell_lines = sorted(set().union(*cell_lines_sets.values()))

# Création du DataFrame
unique_cell_lines_df = pd.DataFrame({"Cell_Line": all_unique_cell_lines})

# Ajout d'une colonne indiquant la présence de chaque lignée dans chaque fichier
for file in files:
    unique_cell_lines_df[file] = unique_cell_lines_df["Cell_Line"].apply(lambda x: x in cell_lines_sets.get(file, set()))

# Trouver le nombre de lignées cellulaires présentes dans tous les fichiers
if cell_lines_sets:
    common_cell_lines = set.intersection(*cell_lines_sets.values())
    print(f"Nombre de lignées cellulaires présentes dans tous les fichiers : {len(common_cell_lines)}")

# Enregistrement des résultats dans un fichier
unique_cell_lines_df.to_csv("unique_cell_lines_per_file.tsv", sep="\t", index=False)
print("Fichier sauvegardé dans 'unique_cell_lines_per_file.tsv'")


Nombre de lignées cellulaires présentes dans tous les fichiers : 459
Fichier sauvegardé dans 'unique_cell_lines_per_file.tsv'


**VERIFICATION**

In [2]:
# Chargement du fichier généré
loaded_df = pd.read_csv("unique_cell_lines_per_file.tsv", sep="\t", dtype=str)

# Comptabilisation des lignées cellulaires présentes dans tous les fichiers
matching_rows = loaded_df[(loaded_df[files] == "True").all(axis=1)]
matching_count = len(matching_rows)

# Affichage du résultat
print(f"Nombre de lignées cellulaires présentes dans tous les fichiers selon le fichier sauvegardé : {matching_count}")

Nombre de lignées cellulaires présentes dans tous les fichiers selon le fichier sauvegardé : 459


**MATCHS AVEC DONNÉES PHENOTYPIQUES CCLE**

In [3]:
# Chargement du fichier des annotations
annotations_df = pd.read_csv(r"C:\\Users\\camil\\IMAG\\CJC\\data\\Cell_lines_annotations_20181226.csv", dtype=str)

# Comparaison entre 'CCLE_ID' et 'Cell_Line'
matches = loaded_df['Cell_Line'].isin(annotations_df['Name'])
match_count = matches.sum()

# Affichage du résultat
print(f"Nombre de correspondances entre 'CCLE_ID' et 'Cell_Line' : {match_count}")

Nombre de correspondances entre 'CCLE_ID' et 'Cell_Line' : 999


In [4]:
# Filtrage des lignées cellulaires correspondantes
matched_cell_lines = loaded_df[matches][['Cell_Line']]

# Sélection des colonnes d'intérêt
columns_of_interest = ["Name", "Pathology", "Site_Primary", "Histology", "Hist_Subtype1", "type", "type_refined", "tcga_code", "Gender", "Age", "Race", "Disease"]

# Fusion des données
final_df = matched_cell_lines.merge(annotations_df[columns_of_interest], left_on='Cell_Line', right_on='Name', how='left').drop(columns=['Name'])

# Suppression des colonnes de présence des fichiers sources
final_df = final_df.drop(columns=[col for col in files if col in final_df.columns], errors='ignore')

# Enregistrement des résultats dans un fichier
final_df.to_csv("matched_cell_lines_annotations.tsv", sep="\t", index=False)